In [159]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [273]:
base_url = 'https://www.runrocknroll.com/'
race_pages = [
        'Events/Nashville/The-Races/Marathon/2019-Results',
        'Events/Nashville/The-Races/Marathon/2018-Results',
        'Events/Nashville/The-Races/Marathon/2017-Results',
        'Events/Nashville/The-Races/Marathon/2016-Results',
        'Events/Nashville/The-Races/Half-Marathon/2019-Results',
        'Events/Nashville/The-Races/Half-Marathon/2018-Results',
        'Events/Nashville/The-Races/Half-Marathon/2017-Results',
        'Events/Nashville/The-Races/Half-Marathon/2016-Results'
    ]
query_parameters = '?gender=&agegroup=&bib=&firstname=&lastname=&page='

In [254]:
def table_append(table):
    """This function takes a soup.find object, iterates through the table lines and adds the
    data in the table to a list"""
    for tr in table.findAll('tr'):
            table_row = []
            for td in tr.findAll('td'):
                if td == None:
                    pass
                else:    
                    table_row.append(td.text)
            data.append(table_row)

In [274]:
# To write each race in a csv in a 'data' folder at the same level as the notebook

possible_pages = range(999)

for race_url in race_pages:
    
    data = []

    for i in possible_pages:
        resp = requests.post(base_url + race_url + query_parameters + str(i+1))
        soup = BeautifulSoup(resp.text, 'html.parser')
        table = soup.find('table', {'class' : 'table'})
        if soup.find('li', {'class': 'PagedList-skipToNext'}).find('a').get('href') == None:
            table_append(table)
            break
        else:
            table_append(table)

    race_df = pd.DataFrame(data)
    race_df.columns = ['Overall', 'Bib', 'Name', 'Time']
    race_df = race_df.dropna(subset = ['Time'])
    race = race_url.split('/')[-2] + '-' + race_url.split('/')[-1]
    
    race_df.to_csv(f'data/{race}.csv')

In [280]:
# To write each race in an excel sheet in a workbook in a 'data' folder at the same level as the notebook

with pd.ExcelWriter('data/marathon_data.xlsx') as writer:

    possible_pages = range(999)

    for race_url in race_pages:
    
        data = []

        for i in possible_pages:
            resp = requests.post(base_url + race_url + query_parameters + str(i+1))
            soup = BeautifulSoup(resp.text, 'html.parser')
            table = soup.find('table', {'class' : 'table'})
            if soup.find('li', {'class': 'PagedList-skipToNext'}).find('a').get('href') == None:
                table_append(table)
                break
            else:
                table_append(table)

        race_df = pd.DataFrame(data)
        race_df.columns = ['Overall', 'Bib', 'Name', 'Time']
        race_df = race_df.dropna(subset = ['Time'])
        race = race_url.split('/')[-2] + '-' + race_url.split('/')[-1]
    
        race_df.to_excel(writer, index=False, sheet_name = f'{race}')